In [7]:
import numpy as np
import pandas as pd

In [31]:
num_covariates = 10

def simulWeib(N):
    # Covariates
    X = np.random.choice([0,1], size=[N, num_covariates])
    
    # Weibull latent event times
    scale1 = 20 + 5*(np.sum(X, axis=1))
    ltime1 = np.random.weibull(scale1, size=N)
    
    scale0 = np.exp(3 + 0.1*(np.sum(X, axis=1)))
    ltime0 = np.random.weibull(scale0, size=N)
    
    # Censoring times
    rate = 0.007
    c1 = np.random.exponential(scale=1/rate, size=N)
    c0 = np.random.exponential(scale=1/rate, size=N)
    
    # Follow-up times and event indicators
    time1 = np.minimum(ltime1, c1) + np.random.uniform(size=N)
    time0 = np.minimum(ltime0, c0) + np.random.uniform(size=N)
    
    status1 = (ltime1 <= c1).astype(int)
    status0 = (ltime0 <= c0).astype(int)
    
    # Treatment assignments 
    T = np.random.choice([0,1], size=N)
    
    # Survival time
    time = np.zeros(N)
    time[T == 1] = time1[T == 1]
    time[T == 0] = time0[T == 0]
    
    # Status
    status = np.zeros(N)
    status[T == 1] = status1[T == 1]
    status[T == 0] = status0[T == 0]
    
    # Create dataframes
    X_df = pd.DataFrame(X, columns=['X{}'.format(i) for i in range(num_covariates)])
    T_df = pd.DataFrame(T, columns=['Treatment'])
    status_df = pd.DataFrame(status, columns=['Status'])
    time_df = pd.DataFrame(time, columns=['Survival Time'])
    
    data = pd.concat([X_df, T_df, status_df, time_df], axis=1)
    return data

In [33]:
data = simulWeib(20)

data

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,Treatment,Status,Survival Time
0,0,0,1,0,1,0,1,1,0,1,1,1.0,1.500180
1,0,1,1,0,1,0,0,1,0,1,0,1.0,1.713588
2,0,0,1,0,0,0,0,1,1,1,1,1.0,1.189391
3,1,1,1,1,1,0,0,1,1,0,0,1.0,1.794512
4,1,1,1,0,0,1,1,1,1,1,1,1.0,1.324048
5,0,1,0,1,0,1,0,1,0,0,1,1.0,1.590667
6,0,1,1,1,0,0,1,1,0,0,0,1.0,1.406061
7,1,0,1,1,0,1,1,0,0,0,0,1.0,1.733967
8,1,0,1,0,1,0,0,0,0,1,0,1.0,1.433346
9,1,0,0,0,1,1,0,0,0,0,0,1.0,1.312276


In [ ]:
simulWeib.nonlin <- function(N)
{set.seed(100)
  # covariates
  X <- matrix(sample(c(0,1),size=4*N, replace=T),ncol=4)
  
  # Weibull latent event times
  Ltime1 <- rweibull(n=N, scale=20 + 5*(X[,1] + X[,2] + X[,3] + X[,4]), shape=1)
  Ltime0 <- rweibull(n=N, scale=exp(3+0.1*(X[,1] + X[,2] + X[,3]) + X[,4]), shape=2)
  
  # censoring times
  C1 <- rexp(n=N, rate=0.007)
  C0 <- rexp(n=N, rate=0.007)
  #C1 <- rweibull(n=N, scale=23 + 4.5*(X[,1] + X[,2] + X[,3] + 10*X[,4]), shape=0.7+1.3*X[,4])
  #C0 <- rweibull(n=N, scale=28 + 4.5*(X[,1] + X[,2] + X[,3] + 10*X[,4]), shape=0.7+1.3*X[,4])
  
  # follow-up times and event indicators
  time1 <- pmin(Ltime1, C1) + runif(N)
  time0 <- pmin(Ltime0, C0) + runif(N)
  status1 <- as.numeric(Ltime1 <= C1)
  status0 <- as.numeric(Ltime0 <= C0)
  
  # treatment assignment
  propensity <- 1/(1+exp(-0.5*(X[,1]+X[,2]+X[,3]+X[,4])))
  treatment = numeric(N)
  for (i in 1:N){
    treatment[i] = sample(c(1,0),size=1,replace=T,prob=c(propensity[i]-0.25, 1.25 - propensity[i]))}
  
  time = numeric(N)
  time[which(treatment == 1)] = time1[which(treatment == 1)]
  time[which(treatment == 0)] = time0[which(treatment == 0)]
  
  status = numeric(N)
  status[which(treatment == 1)] = status1[which(treatment == 1)]
  status[which(treatment == 0)] = status0[which(treatment == 0)]
  
  # data set
  data.frame(id=1:N,
             time=ceiling(time),status=status, treatment=treatment,
             X=X)
}

data <- simulWeib.nonlin(300)